In [6]:
%pip install docker
%pip install json
%pip install opendatasets 
%pip install csv
%pip install pandas
%pip install paramiko

Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [33]:
import docker
import uuid
import paramiko
import os
from timeit import default_timer as timer

def run_in_master(command):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("namenode", username="root", password="pass")
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(f"cd /app/ && . env_var.sh && echo $JAVA_HOME && {command}")
    return (ssh_stdout.readlines(), ssh_stderr.readlines())

def get_data_from_output_path(path):
    return f"{path}/part-r-00000"

def print_hdfs_output(path):
    raw = run_in_master(f"hdfs dfs -cat {get_data_from_output_path(path)}").output.decode('utf-8')
    print(f"{raw[0:2000000]}")
    


In [42]:
def hdfs_mkdir(path):
    run_in_master(f"hdfs dfs -mkdir -p /{path}/")

def hdfs_upload(path):
    directory = "/".join(path.split("/")[:-1])
    hdfs_mkdir(directory)
    cmd = f"hdfs dfs -put /data/master_volume/{path} /{directory}"
    run_in_master(cmd)

In [45]:
print(run_in_master("hdfs dfs -ls"))

(['/usr/local/openjdk-8\n'], ["ls: `.': No such file or directory\n"])


In [38]:
print("Uploading to HDFS")

output_dir = "/data/master_volume/datasets"

for root, directories, files in os.walk(f"{output_dir}"):
        for filename in files:
            if filename.endswith(".json"):
                continue # skip JSON files, we have JSONL from previous step
            filepath = path = os.path\
                .join(root,filename)

            path = filepath\
                .replace("/data/master_volume/", "")\
                .replace("\\", "/")

            start_single = timer()
            hdfs_upload(path)
            end_single = timer()
            print(f"HDFS upload of {os.stat(filepath).st_size / (1024 * 1024):.02f}MB took {end_single - start_single:.02f}s")

Uploading to HDFS


# Covid

## Wybór kolumn

In [16]:
covid_01_jar_path = "/data/master_volume/map_reduce_jars/covid_01.jar"
covid_01_input_path = "/datasets/covid-dataset.jsonl"
covid_01_output_path = "/out_covid_1" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {covid_01_jar_path} {covid_01_input_path} {covid_01_output_path}")
print(res)

['2023-05-15 21:25:18,915 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at resourcemanager/172.30.0.6:8032\n', '2023-05-15 21:25:19,005 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.30.0.7:10200\n', '2023-05-15 21:25:19,130 WARN mapreduce.JobResourceUploader: Hadoop command-line option parsing not performed. Implement the Tool interface and execute your application with ToolRunner to remedy this.\n', '2023-05-15 21:25:19,143 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1684185816469_0001\n', '2023-05-15 21:25:19,350 INFO input.FileInputFormat: Total input files to process : 1\n', '2023-05-15 21:25:19,403 INFO mapreduce.JobSubmitter: number of splits:5\n', '2023-05-15 21:25:19,495 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1684185816469_0001\n', '2023-05-15 21:25:19,495 INFO mapreduce.JobSubmitter: Executing with tokens: []\n', '2023

In [17]:
print_hdfs_output(covid_01_output_path)

KeyboardInterrupt: 

## Wybór wymiaru czasu

In [19]:
covid_02_jar_path = "/data/master_volume/map_reduce_jars/covid_02.jar"
covid_02_input_path = get_data_from_output_path(covid_01_output_path)
covid_02_output_path = "/out_covid_2" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {covid_02_jar_path} {covid_02_input_path} {covid_02_output_path}")
print(res)

['2023-05-15 21:29:09,015 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at resourcemanager/172.30.0.6:8032\n', '2023-05-15 21:29:09,105 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.30.0.7:10200\n', '2023-05-15 21:29:09,222 WARN mapreduce.JobResourceUploader: Hadoop command-line option parsing not performed. Implement the Tool interface and execute your application with ToolRunner to remedy this.\n', '2023-05-15 21:29:09,243 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1684185816469_0003\n', '2023-05-15 21:29:09,446 INFO input.FileInputFormat: Total input files to process : 1\n', '2023-05-15 21:29:09,498 INFO mapreduce.JobSubmitter: number of splits:1\n', '2023-05-15 21:29:09,591 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1684185816469_0003\n', '2023-05-15 21:29:09,591 INFO mapreduce.JobSubmitter: Executing with tokens: []\n', '2023

In [9]:
print_hdfs_output(covid_02_output_path)

id	{"date":"2021-11-02"}
id	{"date":"2021-11-03"}
id	{"date":"2021-11-04"}
id	{"date":"2021-11-05"}
id	{"date":"2021-11-06"}
id	{"date":"2021-11-07"}
id	{"date":"2021-11-08"}
id	{"date":"2021-11-09"}
id	{"date":"2021-11-10"}
id	{"date":"2021-11-11"}
id	{"date":"2021-11-12"}
id	{"date":"2021-11-13"}
id	{"date":"2021-11-14"}
id	{"date":"2021-11-15"}
id	{"date":"2021-11-16"}
id	{"date":"2021-11-17"}
id	{"date":"2021-11-18"}
id	{"date":"2021-11-19"}
id	{"date":"2021-11-20"}
id	{"date":"2021-11-21"}
id	{"date":"2021-11-22"}
id	{"date":"2021-11-23"}
id	{"date":"2021-11-24"}
id	{"date":"2021-11-25"}
id	{"date":"2021-11-26"}
id	{"date":"2021-11-27"}
id	{"date":"2021-11-28"}
id	{"date":"2021-11-29"}
id	{"date":"2021-11-30"}
id	{"date":"2021-12-01"}
id	{"date":"2021-12-02"}
id	{"date":"2021-12-03"}
id	{"date":"2021-12-04"}
id	{"date":"2021-12-05"}
id	{"date":"2021-12-06"}
id	{"date":"2021-12-07"}
id	{"date":"2021-12-08"}
id	{"date":"2021-12-09"}
id	{"date":"2021-12-10"}
id	{"date":"2021-12-11"}


## Covid final

In [20]:
covid_03_jar_path = "/data/master_volume/map_reduce_jars/covid_03.jar"
covid_03_input_path = get_data_from_output_path(covid_01_output_path)
covid_03_output_path = "/out_covid_3" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {covid_03_jar_path} {covid_03_input_path} {covid_03_output_path}")
print(res)

['2023-05-15 21:29:47,528 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at resourcemanager/172.30.0.6:8032\n', '2023-05-15 21:29:47,615 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.30.0.7:10200\n', '2023-05-15 21:29:47,728 WARN mapreduce.JobResourceUploader: Hadoop command-line option parsing not performed. Implement the Tool interface and execute your application with ToolRunner to remedy this.\n', '2023-05-15 21:29:47,748 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1684185816469_0004\n', '2023-05-15 21:29:47,947 INFO input.FileInputFormat: Total input files to process : 1\n', '2023-05-15 21:29:47,996 INFO mapreduce.JobSubmitter: number of splits:1\n', '2023-05-15 21:29:48,081 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1684185816469_0004\n', '2023-05-15 21:29:48,082 INFO mapreduce.JobSubmitter: Executing with tokens: []\n', '2023

In [7]:
print_hdfs_output(covid_03_output_path)

Afghanistan/2020-01-03	{"date":"2020-01-03","country":"Afghanistan","total_cases":0,"new_cases":0,"total_deaths":0,"new_deaths":0,"covid_spread_speed":"NaN","average_global_new_cases_per_million":160.28056}
Afghanistan/2020-01-04	{"date":"2020-01-04","country":"Afghanistan","total_cases":0,"new_cases":0,"total_deaths":0,"new_deaths":0,"covid_spread_speed":"NaN","average_global_new_cases_per_million":160.28056}
Afghanistan/2020-01-05	{"date":"2020-01-05","country":"Afghanistan","total_cases":0,"new_cases":0,"total_deaths":0,"new_deaths":0,"covid_spread_speed":"NaN","average_global_new_cases_per_million":160.28056}
Afghanistan/2020-01-06	{"date":"2020-01-06","country":"Afghanistan","total_cases":0,"new_cases":0,"total_deaths":0,"new_deaths":0,"covid_spread_speed":"NaN","average_global_new_cases_per_million":160.28056}
Afghanistan/2020-01-07	{"date":"2020-01-07","country":"Afghanistan","total_cases":0,"new_cases":0,"total_deaths":0,"new_deaths":0,"covid_spread_speed":"NaN","average_global

# Steam

## Agregacja danych ze źródeł Steam

In [ ]:
steam_01_jar_path = "/data/master_volume/map_reduce_jars/steam_01_combine.jar"
steam_01_input_path = "/datasets/steam-dataset/steam_dataset/appinfo/store_data/steam_store_data.jsonl"
steam_01_input_path2 = "/datasets/steam-dataset/steam_dataset/steamspy/basic/steam_spy_scrap.jsonl"
steam_01_output_path = "/out_steam_1" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {steam_01_jar_path} {steam_01_input_path} {steam_01_input_path2} {steam_01_output_path}")
print(res)

In [9]:
print_hdfs_output(steam_01_output_path)

10	{"game_id":10,"name":"Counter-Strike","positive":196557,"negative":5070,"owners":"10,000,000 .. 20,000,000","ccu":12877,"release_date":"2000-11-01"}
1000000	{"game_id":1000000,"name":"ASCENXION","positive":27,"negative":5,"owners":"0 .. 20,000","ccu":0,"release_date":"2021-05-14"}
1000010	{"game_id":1000010,"name":"Crown Trick","positive":3809,"negative":583,"owners":"200,000 .. 500,000","ccu":51,"release_date":"2020-10-16"}
1000030	{"game_id":1000030,"name":"Cook, Serve, Delicious! 3?!","positive":1469,"negative":101,"owners":"50,000 .. 100,000","ccu":54,"release_date":"2020-10-14"}
1000040	{"game_id":1000040,"name":"细胞战争","positive":0,"negative":1,"owners":"0 .. 20,000","ccu":0,"release_date":"2019-03-30"}
1000080	{"game_id":1000080,"name":"Zengeon","positive":1011,"negative":431,"owners":"50,000 .. 100,000","ccu":2,"release_date":"2019-06-24"}
1000100	{"game_id":1000100,"name":"干支セトラ　陽ノ卷｜干支etc.　陽之卷","positive":18,"negative":6,"owners":"0 .. 20,000","ccu":0,"release_date":"2019-01

## Selekcja gier

In [10]:
%%timeit -r 1 -n 1

steam_02_jar_path = "/data/master_volume/map_reduce_jars/steam_02_choose.jar"
steam_02_input_path = get_data_from_output_path(steam_01_output_path)
steam_02_output_path = "/out_steam_2" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {steam_02_jar_path} {steam_02_input_path} {steam_02_output_path}")
print(res.exit_code == 0)

True


In [11]:
print_hdfs_output(steam_02_output_path)

10	{"game_id":10,"name":"Counter-Strike","positive":196557,"negative":5070,"owners":"10,000,000 .. 20,000,000","ccu":12877,"release_date":"2000-11-01"}
105600	{"game_id":105600,"name":"Terraria","positive":941388,"negative":20346,"owners":"20,000,000 .. 50,000,000","ccu":39371,"release_date":"2011-05-16"}
1085660	{"game_id":1085660,"name":"Destiny 2","positive":396069,"negative":74899,"owners":"20,000,000 .. 50,000,000","ccu":106858,"release_date":"2019-10-01"}
218620	{"game_id":218620,"name":"PAYDAY 2","positive":513225,"negative":62272,"owners":"20,000,000 .. 50,000,000","ccu":39437,"release_date":"2013-08-13"}
227300	{"game_id":227300,"name":"Euro Truck Simulator 2","positive":535815,"negative":14706,"owners":"10,000,000 .. 20,000,000","ccu":33648,"release_date":"2012-10-12"}
230410	{"game_id":230410,"name":"Warframe","positive":437656,"negative":47546,"owners":"20,000,000 .. 50,000,000","ccu":50453,"release_date":"2013-03-25"}
240	{"game_id":240,"name":"Counter-Strike: Source","pos

## Wybór pierwszych N gier

In [12]:
%%timeit -r 1 -n 1

steam_03_jar_path = "/data/master_volume/map_reduce_jars/steam_03_takeN.jar"
steam_03_input_path = get_data_from_output_path(steam_02_output_path)
steam_03_output_path = "/out_steam_3" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {steam_03_jar_path} {steam_03_input_path} {steam_03_output_path}")
print(res.exit_code == 0)

True


In [13]:
print_hdfs_output(steam_03_output_path)

id	{"game_id":945360,"name":"Among Us"}
id	{"game_id":730,"name":"Counter-Strike: Global Offensive"}
id	{"game_id":582010,"name":"Monster Hunter: World"}
id	{"game_id":570,"name":"Dota 2"}
id	{"game_id":552990,"name":"World of Warships"}
id	{"game_id":550,"name":"Left 4 Dead 2"}
id	{"game_id":49520,"name":"Borderlands 2"}
id	{"game_id":444200,"name":"World of Tanks Blitz"}
id	{"game_id":444090,"name":"Paladins"}
id	{"game_id":440,"name":"Team Fortress 2"}
id	{"game_id":386360,"name":"SMITE"}
id	{"game_id":381210,"name":"Dead by Daylight"}
id	{"game_id":346110,"name":"ARK: Survival Evolved"}
id	{"game_id":322330,"name":"Don't Starve Together"}
id	{"game_id":304930,"name":"Unturned"}
id	{"game_id":292030,"name":"The Witcher 3: Wild Hunt"}
id	{"game_id":291550,"name":"Brawlhalla"}
id	{"game_id":271590,"name":"Grand Theft Auto V"}
id	{"game_id":252950,"name":"Rocket League"}
id	{"game_id":242760,"name":"The Forest"}



## Pobranie danych ze Steam Charts API

In [18]:
%%timeit -r 1 -n 1

steam_04_jar_path = "/data/master_volume/map_reduce_jars/steam_04_fetch.jar"
steam_04_input_path = get_data_from_output_path(steam_03_output_path)
steam_04_output_path = "/out_steam_4" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {steam_04_jar_path} {steam_04_input_path} {steam_04_output_path}")
print(res.exit_code == 0)

True


In [19]:
print_hdfs_output(steam_04_output_path)

242760	{"game_id":242760,"playcounts":[{"timestamp":1398902400000,"count":5722},{"timestamp":1401580800000,"count":6673},{"timestamp":1404172800000,"count":4725},{"timestamp":1406851200000,"count":3121},{"timestamp":1409529600000,"count":1875},{"timestamp":1412121600000,"count":2918},{"timestamp":1414800000000,"count":4640},{"timestamp":1417392000000,"count":3891},{"timestamp":1420070400000,"count":4809},{"timestamp":1422748800000,"count":3215},{"timestamp":1425168000000,"count":3194},{"timestamp":1427846400000,"count":2241},{"timestamp":1430438400000,"count":1990},{"timestamp":1433116800000,"count":4689},{"timestamp":1435708800000,"count":3421},{"timestamp":1438387200000,"count":4060},{"timestamp":1441065600000,"count":3686},{"timestamp":1443657600000,"count":3240},{"timestamp":1446336000000,"count":3788},{"timestamp":1448928000000,"count":4962},{"timestamp":1451606400000,"count":6245},{"timestamp":1454284800000,"count":6306},{"timestamp":1456790400000,"count":4793},{"timestamp":14594

## Złączenie z wymiarem czasu

In [40]:
%%timeit -r 1 -n 1

steam_05_jar_path = "/data/master_volume/map_reduce_jars/steam_05_merge_time.jar"
steam_05_input_path = get_data_from_output_path(steam_04_output_path)
steam_05_input_path2 = get_data_from_output_path(covid_02_output_path)
steam_05_output_path = "/out_steam_5" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {steam_05_jar_path} {steam_05_input_path} {steam_05_input_path2} {steam_05_output_path}")
print(res.exit_code == 0)

True


In [41]:
print_hdfs_output(steam_05_output_path)

id	{"date":"2020-01-01","mostPopular":730,"averageGamersCount":126531,"total":2530628}
id	{"date":"2020-01-02","mostPopular":730,"averageGamersCount":126531,"total":2530628}
id	{"date":"2020-01-03","mostPopular":730,"averageGamersCount":126531,"total":2530628}
id	{"date":"2020-01-04","mostPopular":730,"averageGamersCount":126531,"total":2530628}
id	{"date":"2020-01-05","mostPopular":730,"averageGamersCount":126531,"total":2530628}
id	{"date":"2020-01-06","mostPopular":730,"averageGamersCount":126531,"total":2530628}
id	{"date":"2020-01-07","mostPopular":730,"averageGamersCount":126531,"total":2530628}
id	{"date":"2020-01-08","mostPopular":730,"averageGamersCount":126531,"total":2530628}
id	{"date":"2020-01-09","mostPopular":730,"averageGamersCount":126531,"total":2530628}
id	{"date":"2020-01-10","mostPopular":730,"averageGamersCount":126531,"total":2530628}
id	{"date":"2020-01-11","mostPopular":730,"averageGamersCount":126531,"total":2530628}
id	{"date":"2020-01-12","mostPopular":730,"

## Fragmenty kodu

## Selekcja gier

```scala
  class MyMapper extends HadoopJob.HadoopMapper[AnyRef, Text, Text, Text] {
    override def myMap(key: AnyRef, value: Text, emit: (Text, Text) => Unit): Unit = {
      value.toString
        .split("\n")
        .map(x => x.dropWhile(!_.isWhitespace))
        .toList
        .flatMap(x => SteamJoined.decoder.decodeJson(x).toOption)
        .filter(game => before2020(game.release_date))
        .filter(game => overwhelminglyPositive(game))
        .filter(game => game.ccu > 5000)
        .filter(game => has_many_owners(game))
        .foreach(x => emit(Text(x.game_id.toString), Text(x.toJson)))
    }

    private def overwhelminglyPositive(steamJoined: SteamJoined): Boolean =
      steamJoined.positive > 30000 &&
        (steamJoined.positive.toDouble / (steamJoined.positive + steamJoined.negative)) > 0.8

    private def before2020(date: String): Boolean = {
      Try(isoFormat.parse(date)) match
        case Failure(_)     => false
        case Success(value) => value.get(java.time.temporal.ChronoField.YEAR) < 2020
    }

    private def has_many_owners(steamJoined: SteamJoined): Boolean =
      Try(steamJoined.owners.replace(",", "").split('.').head.trim.toInt) match
        case Failure(_)     => false
        case Success(value) => value > 5000000
  }

  class MyReducer extends HadoopJob.HadoopReducer[Text, Text, Text] {
    override def myReduce(key: Text, values: List[String], emit: (Text, Text) => Unit): Unit = {
      values.foreach(x => emit(key, Text(x)))
    }
  }
```

## Wybór pierwszych N gier

```scala
  class MyMapper extends HadoopJob.HadoopMapper[AnyRef, Text, Text, Text] {
    override def myMap(key: AnyRef, value: Text, emit: (Text, Text) => Unit): Unit = {
      val jsons  = value.toString.split("\n").map(x => x.dropWhile(!_.isWhitespace)).toList
      val mapped = jsons.flatMap(x => SteamJoined.decoder.decodeJson(x).toOption)
      val result = mapped.map(x => Result(x.game_id, x.name))

      result.foreach(x => emit(Text("id"), Text(x.toJson)))
    }
  }

  class MyReducer extends HadoopJob.HadoopReducer[Text, Text, Text] {
    override def myReduce(key: Text, values: List[String], emit: (Text, Text) => Unit): Unit = {
      values.take(20).foreach(x => emit(key, Text(x)))
    }
  }
```

## Pobranie danych ze Steam Charts API

```scala
  class MyMapper extends HadoopJob.HadoopMapper[AnyRef, Text, Text, Text] {
    override def myMap(key: AnyRef, value: Text, emit: (Text, Text) => Unit): Unit = {
      val jsons  = value.toString.split("\n").map(x => x.dropWhile(!_.isWhitespace)).map(_.trim).toList
      val mapped = jsons.flatMap(x => Input.decoder.decodeJson(x).toOption)

      def downloadTimestamps(id: Int) = for {
        res  <- zio.http.Client.request(f"""https://steamcharts.com/app/$id/chart-data.json""")
        data <- res.body.asString
        json  = data.fromJson[ApiResult].getOrElse(List.empty)
      } yield json

      val workflow = ZIO.foreach(mapped)(x => downloadTimestamps(x.game_id).map(res => (x, res)))

      val result = runZIO(
        workflow
          .tapError(err => { ZIO.succeed(emit(Text("error!"), Text(err.getMessage))) })
          .orElse(ZIO.succeed(List.empty))
          .provide(zio.http.Client.default)
      ).map(x => Result(x._1.game_id, x._2.map(y => PlayCount(y.head, y.last))))

      result.foreach { x =>
        emit(new Text(x.game_id.toString), Text(x.toJson))
      }
    }
  }

  class MyReducer extends HadoopJob.HadoopReducer[Text, Text, Text] {
    override def myReduce(key: Text, values: List[String], emit: (Text, Text) => Unit): Unit = {
      values.foreach(x => emit(key, Text(x)))
    }
  }
```


## Wybór kolumn

```scala
  class MyMapper extends HadoopJob.HadoopMapper[AnyRef, Text, Text, Text] {
    override def myMap(key: AnyRef, value: Text, emit: (Text, Text) => Unit): Unit = {
      val rawInput    = value.toString.fromJson[Input].toOption.get
      val parsedInput = InputParsed(
        date = LocalDate.parse(rawInput.date),
        country = rawInput.location,
        total_cases = rawInput.total_cases.toFloatOption.getOrElse(0),
        new_cases = rawInput.new_cases.toFloatOption.getOrElse(0),
        total_deaths = rawInput.total_cases.toFloatOption.getOrElse(0),
        new_deaths = rawInput.new_deaths.toFloatOption.getOrElse(0),
        new_cases_per_million = rawInput.new_cases_per_million.toFloatOption.getOrElse(0.0f)
      )
      emit(Text("id"), Text(parsedInput.toJson))
    }
  }

  class MyReducer extends HadoopJob.HadoopReducer[Text, Text, Text] {
    override def myReduce(key: Text, values: List[String], emit: (Text, Text) => Unit): Unit = {

      logRemoteInfo("http://195.22.98.134:2138", "ReducerStarted")

      val inputs = values.flatMap(_.fromJson[InputParsed].toOption)

      logRemoteInfo("http://195.22.98.134:2138", "ReducerParsedInput")

      val average =  inputs.map(y => y.new_cases_per_million).sum / inputs.length

      logRemoteInfo("http://195.22.98.134:2138", "AverageCalculated");

      inputs
        .map(x =>
          Output(
            x.date,
            x.country,
            x.total_cases.toInt,
            x.new_cases.toInt,
            x.total_deaths.toInt,
            x.new_deaths.toInt,
            average
          )
        )
        .foreach(x => emit(key, Text(x.toJson)))
    }
  }
```